# Scrape Exchange Rate

This notebook scrapes the daily USD-MNT exchange rate from the MongolBank website. 

In [1]:
from bs4 import BeautifulSoup
import requests
import datetime
import pandas as pd

In [2]:
start = datetime.datetime(2000,1,1)
end = datetime.datetime(2020,4,14)
delta = end - start

In [3]:
days = []
urls = []

In [4]:
for i in range(delta.days + 1):
    days.append(start + datetime.timedelta(days=i))

In [5]:
exchange_rate = pd.DataFrame(columns=['Date','USD'])

In [6]:
for day in days:
    urls.append(f'https://www.mongolbank.mn/eng/dblistofficialdailyrate.aspx?vYear={str(day.year)}&vMonth={str(day.month)}&vDay={str(day.day)}')

In [7]:
scraping = pd.DataFrame(days, columns=['days'])
scraping['urls'] = urls

In [8]:
scraping.head()

,days,urls
0,2000-01-01,https://www.mongolbank.mn/eng/dblistofficialda...
1,2000-01-02,https://www.mongolbank.mn/eng/dblistofficialda...
2,2000-01-03,https://www.mongolbank.mn/eng/dblistofficialda...
3,2000-01-04,https://www.mongolbank.mn/eng/dblistofficialda...
4,2000-01-05,https://www.mongolbank.mn/eng/dblistofficialda...


In [11]:
scraping.iloc[0]['urls']

'https://www.mongolbank.mn/eng/dblistofficialdailyrate.aspx?vYear=2000&vMonth=1&vDay=1'

In [ ]:
for index, row in scraping.iterrows():
    try:
        #fetch the page from the URL
        page_response = requests.get(row['urls'], timeout=15)

        #Initialize BeautifulSoup and parse page data
        soup = BeautifulSoup(page_response.content)
        
        #find an store variables
        usd = soup.find(id="ContentPlaceHolder1_lblUSD").text
        date = row['days'].strftime('%m/%d/%Y')
        #append scraped data to dataframe
        exchange_rate = exchange_rate.append({'Date': date,
                                              'USD': usd}, 
                                             ignore_index=True)
    except:
        exchange_rate = exchange_rate.append({'Date': 'error',
                                              'USD': 'error'},
                                             ignore_index=True)    

In [12]:
exchange_rate.head()    

,Date,USD
0,01/01/2000,
1,01/02/2000,
2,01/03/2000,
3,01/04/2000,
4,01/05/2000,


In [ ]:
## Use this for exporting to local project folder when using a local runtime.
exchange_rate.to_csv('./data/raw/usd-mnt.csv', index=False)